In [5]:
# %% Imports
import pandas as pd
import sys
import numpy as np

sys.path.append("../")
from metrics.metric_participants import (ComputeMetrics, print_metrics)
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklego.preprocessing import ColumnSelector
from sktools import IsEmptyExtractor, QuantileEncoder
from lightgbm import LGBMRegressor
from category_encoders import TargetEncoder
import random

from eda.checker import check_train_test
from tools.postprocessing import postprocess_predictions

# read data

In [40]:

random.seed(0)

sales_train = pd.read_csv("../data/data_raw/sales_train.csv")
df_full = pd.read_csv("../data/split.csv")
df_region = pd.read_csv("../data/data_raw/regions.csv")
regions_hcps = pd.read_csv("../data/data_raw/regions_hcps.csv")
activity_features = pd.read_csv("../data/features/activity_features.csv")
brands_3_12 = pd.read_csv("../data/features/brand_3_12_market_features_lagged.csv")
rte_basic = pd.read_csv("../data/features/rte_basic_features.csv").drop(
    columns=["sales", "validation"]
)

market_size = pd.read_csv("../data/market_size.csv")

In [41]:
# For reproducibility
random.seed(0)
VAL_SIZE = 38
SUBMISSION_NAME = "add_market_estimation"
RETRAIN = True

In [42]:
# %% Training weights
market_size = (
    market_size
    .assign(weight=lambda x: 100 / x['sales'])
    .rename(columns={"sales": 'market_size'})
)


# %% Add region data
df_feats = df_full.merge(df_region, on="region", how="left")
df_feats = pd.merge(left=df_feats, right=regions_hcps, how="left", on="region")
df_feats = df_feats.merge(
    activity_features, on=["month", "region", "brand"], how="left"
)
df_feats = df_feats.merge(rte_basic, on=["month", "region", "brand"], how="left")
df_feats = df_feats.merge(brands_3_12, on=["month", "region"], how="left")
df_feats["whichBrand"] = np.where(df_feats.brand == "brand_1", 1, 0)

df_feats = df_feats.merge(market_size, on='region', how="left")

df_feats['month_brand'] = df_feats.month + '_' + df_feats.brand

df_feats['market_estimation'] = (
    df_feats.sales_brand_12_market * df_feats.sales_brand_3
) / df_feats.sales_brand_3_market

df_feats.loc[df_feats.brand == 'brand_1', 'market_estimation'] = 0.75 * df_feats.loc[df_feats.brand == 'brand_1', 'market_estimation']
df_feats.loc[df_feats.brand == 'brand_2', 'market_estimation'] = 0.25 * df_feats.loc[df_feats.brand == 'brand_2', 'market_estimation']

In [50]:
# drop sum variables
cols_to_drop = ["region", "sales", "validation", "market_size", "weight"]
cols_to_drop = []

# %% Split train val test
X_train = df_feats.query("validation == 0").drop(columns=cols_to_drop)
y_train = df_feats.query("validation == 0").sales
weights_train = df_feats.query("validation == 0").weight

X_val = df_feats.query("validation == 1").drop(columns=cols_to_drop)
y_val = df_feats.query("validation == 1").sales

X_full = df_feats.query("validation.notnull()", engine="python").drop(
    columns=cols_to_drop
)
y_full = df_feats.query("validation.notnull()", engine="python").sales
weights_full = df_feats.query("validation.notnull()", engine="python").weight

X_test = df_feats.query("validation.isnull()", engine="python").drop(
    columns=cols_to_drop
)
y_test = df_feats.query("validation.isnull()", engine="python").sales

In [53]:
select_cols = [
    "month_brand",
    "sales_brand_3",
    "inverse_tier_f2f",
    "hcp_distinct_Internal medicine / pneumology",
    "sales_brand_12_market_per_region",
    "sales_brand_12_market",
    'no. openings_Pediatrician',
    'tier_openings_Internal medicine / pneumology',
    'area_x',
    'market_estimation'
]
assert len([col for col in X_train.columns if col in select_cols]) == len(select_cols)

In [54]:
# %%
lgbms = {}
pipes = {}
train_preds = {}
val_preds = {}
test_preds = {}

for quantile in [0.5, 0.1, 0.9]:

    lgbms[quantile] = LGBMRegressor(
        n_jobs=-1,
        n_estimators=50,
        objective="quantile",
        alpha=quantile,
    )

    pipes[quantile] = Pipeline(
        [   
            ("te", TargetEncoder(cols=["month_brand", "month", "brand"])),
            ("selector", ColumnSelector(columns=select_cols)),
            ("imputer", SimpleImputer(strategy="median", add_indicator=True)), 
            ("lgb", lgbms[quantile])
        ]
    )

    # Fit cv model
    pipes[quantile].fit(X_train, y_train)
    # , lgb__sample_weight=weights_train)

    train_preds[quantile] = pipes[quantile].predict(X_train)
    val_preds[quantile] = pipes[quantile].predict(X_val)


    if RETRAIN:
        pipes[quantile].fit(X_full, y_full)
        # , lgb__sample_weight=weights_full)
    test_preds[quantile] = pipes[quantile].predict(X_test)

In [74]:
test = X_test.fillna(0)
test_regions = test['region'].unique()

In [103]:
results = {}
inc = 0.05
for region in test_regions:
    x = test.query(f"region=='{region}'").copy()
    before = sum(pipes[0.5].predict(x))
    x["hcp_distinct_Internal medicine / pneumology"] = x["hcp_distinct_Internal medicine / pneumology"] * (1+inc)
    after = sum(pipes[0.5].predict(x))
    results[region] = [100*(after - before) / before]
r = pd.DataFrame(results).T.reset_index()
r.columns = ['region', 'increase']
r.sort_values('increase', ascending=False).head()

,region,increase
5,region_156,7.619022
29,region_180,2.987129
1,region_152,1.998953
33,region_184,1.287994
40,region_191,0.571545


In [104]:
results = {}
inc = 0.05
for region in test_regions:
    x = test.query(f"region=='{region}'").copy()
    before = sum(pipes[0.5].predict(x))
    x["inverse_tier_f2f"] = x["inverse_tier_f2f"] * (1+inc)
    after = sum(pipes[0.5].predict(x))
    results[region] = [100*(after - before) / before]
r = pd.DataFrame(results).T.reset_index()
r.columns = ['region', 'increase']
r.sort_values('increase', ascending=False).head()

,region,increase
15,region_166,5.776660
37,region_188,5.614231
0,region_151,4.353752
41,region_192,3.629485
16,region_167,3.076168


In [65]:
x

,brand,region,month_brand,sales_brand_3,inverse_tier_f2f,hcp_distinct_Internal medicine / pneumology,sales_brand_12_market_per_region,sales_brand_12_market,no. openings_Pediatrician,tier_openings_Internal medicine / pneumology,area_x,market_estimation
4326,brand_1,region_200,2020-07_brand_1,65456.48,0.0,0.0,6644712.06,301746.46,0.0,0.0,2598.725436,25062.796112
4327,brand_2,region_200,2020-07_brand_2,65456.48,30.0,0.0,6644712.06,301746.46,0.0,0.0,2598.725436,8354.265371
4426,brand_1,region_200,2020-08_brand_1,60606.39,36.0,2.0,6644712.06,274216.30,0.0,6.0,2598.725436,23455.761267
4427,brand_2,region_200,2020-08_brand_2,60606.39,65.0,0.0,6644712.06,274216.30,0.0,0.0,2598.725436,7818.587089
4526,brand_1,region_200,2020-09_brand_1,67185.15,75.0,2.0,6644712.06,329985.60,0.0,6.0,2598.725436,27469.541743
4527,brand_2,region_200,2020-09_brand_2,67185.15,98.0,0.0,6644712.06,329985.60,0.0,0.0,2598.725436,9156.513914
4626,brand_1,region_200,2020-10_brand_1,64849.61,103.0,3.0,6644712.06,314929.68,0.0,9.0,2598.725436,25722.543271
4627,brand_2,region_200,2020-10_brand_2,64849.61,122.0,0.0,6644712.06,314929.68,0.0,0.0,2598.725436,8574.181090
4726,brand_1,region_200,2020-11_brand_1,64072.92,103.0,5.0,6644712.06,338413.61,0.0,9.0,2598.725436,26450.002100
4727,brand_2,region_200,2020-11_brand_2,64072.92,122.0,0.0,6644712.06,338413.61,0.0,0.0,2598.725436,8816.667367


In [15]:
pipes[0.5]

Pipeline(steps=[('te', TargetEncoder(cols=['month_brand', 'month', 'brand'])),
                ('selector',
                 ColumnSelector(columns=['month_brand', 'sales_brand_3',
                                         'inverse_tier_f2f',
                                         'hcp_distinct_Internal medicine / '
                                         'pneumology',
                                         'sales_brand_12_market_per_region',
                                         'sales_brand_12_market',
                                         'no. openings_Pediatrician',
                                         'tier_openings_Internal medicine / '
                                         'pneumology',
                                         'area_x', 'market_estimation'])),
                ('imputer',
                 SimpleImputer(add_indicator=True, strategy='median')),
                ('lgb',
                 LGBMRegressor(alpha=0.5, n_estimators=50,
                      

In [13]:
# %% Postprocess
train_preds_post = postprocess_predictions(train_preds)
val_preds_post = postprocess_predictions(val_preds)
test_preds_post = postprocess_predictions(test_preds)

In [ ]:
# %% Train prediction
train_preds_post

# %% Train prediction
train_preds_df = (
    df_feats.query("validation == 0")
    .loc[:, ["month", "region", "brand"]]
    .assign(sales=train_preds_post[0.5])
    .assign(lower=train_preds_post[0.1])
    .assign(upper=train_preds_post[0.9])
)

ground_truth_train = df_feats.query("validation == 0").loc[
    :, ["month", "region", "brand", "sales"]
]

print_metrics(train_preds_df, sales_train, ground_truth_train)

# %% Validation prediction
val_preds_df = (
    df_feats.query("validation == 1")
    .loc[:, ["month", "region", "brand"]]
    .assign(sales=val_preds_post[0.5])
    .assign(lower=val_preds_post[0.1])
    .assign(upper=val_preds_post[0.9])
)

ground_truth_val = df_feats.query("validation == 1").loc[
    :, ["month", "region", "brand", "sales"]
]

print_metrics(val_preds_df, sales_train, ground_truth_val)

# %%
val_preds_df.to_csv(f"../data/validation/{SUBMISSION_NAME}_val.csv", index=False)


# %% Test prediction
test_preds_df = (
    df_feats.query("validation.isnull()", engine="python")
    .loc[:, ["month", "region", "brand"]]
    .assign(sales=test_preds_post[0.5])
    .assign(lower=test_preds_post[0.1])
    .assign(upper=test_preds_post[0.9])
)

test_preds_df.to_csv(f"../submissions/{SUBMISSION_NAME}.csv", index=False)